In [1]:
# Feature extraction for sound analysis of birdsong
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa

from scipy.io import wavfile
import scipy.signal as signal
import IPython.display as ipd

%matplotlib inline

In [ ]:
sound_file = '2023_03_31_7_03_30.wav'
fs, audio = wavfile.read(sound_file)
audio_clip = audio[:fs*5]
ipd.Audio(sound_file)

Implemented Features

## STFT and Spectral Derivative

In [ ]:
import scipy.signal as signal
from scipy.fft import fft, ifft, fftfreq, fftshift
window_size = 1323
hop_length = 163

x = audio_clip

tapers = signal.windows.dpss(window_size, 1.5, 2)
size = len(x)
f_notShifted = fftfreq(window_size, 1/fs)
f = fftshift(f_notShifted)
f_index = f > 0

sonogram = np.zeros((f_index.sum(), np.floor(size / hop_length).astype(int)))
freq_deriv = np.zeros((f_index.sum(), np.floor(size / hop_length).astype(int)))
time_deriv = np.zeros((f_index.sum(), np.floor(size / hop_length).astype(int)))
frequency_modulation = np.zeros(np.floor(size / hop_length).astype(int))
spectral_derivative = np.zeros((f_index.sum(), np.floor(size / hop_length).astype(int)))

wav_smp = np.arange(size-window_size, step=hop_length).astype(int)
t = np.arange(np.floor(size / hop_length)) * (hop_length/fs)
for i in range(len(wav_smp)):
    samps = np.arange(wav_smp[i], np.floor(wav_smp[i] + window_size).astype(int))
    window1 = x[samps] * tapers[0]
    window2 = x[samps] * tapers[1]
    
    powSpect1 = fftshift(fft(window1))
    powSpect2 = fftshift(fft(window2))
    r1 = (np.abs(powSpect1) + np.abs(powSpect2))**2
    sonogram[:,i] = r1[f_index]
    
    # Directely from SAP code
    fR1 = np.real(powSpect1[f_index])
    fi1 = np.imag(powSpect1[f_index])
    fR2 = np.real(powSpect2[f_index])
    fi2 = np.imag(powSpect2[f_index])
    
    time_deriv[:,i] = -fR1*fR2 - fi1*fi2
    freq_deriv[:,i] = fi1*fR2 - fR1*fi2
    frequency_modulation[i] = np.arctan((np.max(time_deriv[:,i])/np.max(freq_deriv[:,i]))+0.1)
    
    cFM = np.cos(frequency_modulation[i])
    sFM = np.sin(frequency_modulation[i])
    spectral_derivative[:,i] = time_deriv[:,i].dot(cFM) + freq_deriv[:,i].dot(sFM)

# Plot the sonogram
import matplotlib.pyplot as plt

plt.imshow(np.log(sonogram), aspect='auto', origin='lower')
plt.colorbar()
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.title('Sonogram')
plt.show()

plt.imshow(spectral_derivative/np.max(spectral_derivative), aspect='auto', origin='lower', cmap='bwr')
plt.colorbar()
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.title('Frequency Derivative')
plt.show()

## Amplitude

In [ ]:
amp = 20*np.log10(np.sum(stft, axis=0))-70

plt.clf()
fig, ax1 = plt.subplots(figsize=(20,10))
ax1.pcolormesh(t,f, spec_der)
plt.ylim([0, 10000])
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.title('Amplitude (dB)')

ax2 = ax1.twinx()
ax2.plot(t, amp)

## Pitch Estimation (Peak/Mean Frequency)

In [ ]:
peak_freq = f[np.argmax(stft, axis=0)]
mean_freq = f.dot(stft**2) / np.sum(stft**2, axis=0)

plt.clf()
plt.subplots(figsize=(20,10))
plt.pcolormesh(t,f, spec_der)
plt.ylim([0, 10000])

plt.plot(t, peak_freq, '.k', label='Peak Frequency')
plt.plot(t, mean_freq, '.r', label='Mean Frequency')
plt.title('Pitch Estimatation For Sample Song')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.legend()

## Fundemental Frequency
Using Librosa function

In [ ]:
# Zebra finch range rarely goes over 1800 according to SAP
freq_range = [50, 2000]

ff_lib = librosa.yin(audio_clip.astype(float), fmin=freq_range[0], fmax=freq_range[1], sr=fs)
t_lib = np.linspace(0, len(audio_clip)/fs, len(ff_lib))

plt.clf()
plt.subplots(figsize=(20,10))
plt.pcolormesh(t,f, spec_der)
plt.ylim([0, 10000])

plt.plot(t, peak_freq, '.k', label='Peak Frequency')
plt.plot(t, mean_freq, '.r', label='Mean Frequency')
plt.plot(t_lib, ff_lib, '.b', label='Fundamental Frequency')
plt.title('Pitch Estimatation For Sample Song')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.legend()


# Wiener Entropy

In [7]:
# Taken from antropy package, list of functions for entropy calculation
# https://github.com/raphaelvallat/antropy/tree/master/antropy

def xlogx(x, base=2):
    """Returns x log_b x if x is positive, 0 if x == 0, and np.nan
    otherwise. This handles the case when the power spectrum density
    takes any zero value.
    """
    x = np.asarray(x)
    xlogx = np.zeros(x.shape)
    xlogx[x < 0] = np.nan
    valid = x > 0
    xlogx[valid] = x[valid] * np.log(x[valid]) / np.log(base)
    return xlogx

def spectral_entropy(x, fs, method='fft', nperseg=None, normalize=True, axis=-1):
    x = np.asarray(x)
    # Compute and normalize power spectrum
    if method == "fft":
        _, psd = signal.periodogram(x, fs, axis=axis)
    elif method == "welch":
        _, psd = signal.welch(x, fs, nperseg=nperseg, axis=axis)
    psd_norm = psd / psd.sum(axis=axis, keepdims=True)
    se = -xlogx(psd_norm).sum(axis=axis)
    if normalize:
        se /= np.log2(psd_norm.shape[axis])
    return se

# Create overlapping blocks for Overlap-Add Method
# From https://www.kuniga.me/blog/2021/12/11/pitch-via-cepstrum.html

def create_overlapping_blocks(x, window_length, fs, hop_length):
    n = len(x)
    w = signal.windows.hann(window_length)
    nw = len(w)
    step = hop_length
    nb = np.floor((n - nw) / step).astype(int) + 1

    B = np.zeros((nb, nw))
    t = np.arange(nb) * (step/fs)

    for i in range(nb):
        offset = i * step
        B[i, :] = w * x[offset : nw + offset]

    return B, t

In [ ]:
B, t_ent_extra = create_overlapping_blocks(audio_clip, window_length, fs, hop_length)
spec_ent_xtra = np.apply_along_axis(spectral_entropy, 1, B, fs)

t_ent = np.interp(t, t_ent_extra, t_ent_extra)
spec_ent = np.interp(t, t_ent_extra, spec_ent_xtra)

plt.clf()
fig, ax1 = plt.subplots(figsize=(20,10))
ax1.pcolormesh(t,f, spec_der)
plt.ylim([0, 10000])
ax1.set_yticks([])
plt.xlabel('Time (s)')

ax2 = ax1.twinx()
ax2.plot(t_ent, spec_ent, 'g', label='Spectral Entropy')

ax3 = ax2.twinx()
ax3.plot(t, amp, label='Amplitude')
plt.title('Amplitude (dB) and Spectral Enropy (Unitless)')
plt.tight_layout()

## Frequency Modulation

In [ ]:
freq_comp = np.argmax(dPdt**2, axis=0)
time_comp = np.argmax(dPdf**2, axis=0)
freq_mod = np.arctan(freq_comp / time_comp)

plt.clf()
fig, ax1 = plt.subplots(figsize=(20,10))
plt.pcolormesh(t,f, spec_der)
plt.ylim([0, 10000])
plt.xlabel('Time (s)')
plt.title('Frequency Modulation')

ax2 = ax1.twinx()
ax2.plot(t, freq_mod, 'c')

## Amplitude Modulation

In [ ]:
amp_mod = dPdt.sum(axis=0)

plt.clf()
fig, ax1 = plt.subplots(figsize=(20,10))
ax1.pcolormesh(t,f, spec_der)
plt.ylim([0, 10000])
plt.xlabel('Time (s)') 

ax2 = ax1.twinx()
ax2.plot(t, amp_mod, 'm')
plt.title('Amplitude Modulation')

## Spectral Continuity

In [ ]:
kernel = 10e-6*np.array([[1,1,1], [1, -8, 1], [1,1,1]])
result = signal.convolve2d(spec_der, kernel, mode='same')
T_prime = 100
threshold = np.zeros_like(result)

for f_idx, temp_f in enumerate(f):
    for t_idx, temp_t in enumerate(t):
        threshold[f_idx, t_idx] = T_prime*np.abs(spec_ent[t_idx]) / np.abs(temp_f - mean_freq[t_idx])

contour = result > threshold

plt.clf()
plt.subplots(figsize=(20,10))
plt.subplot(2,1,1)
plt.pcolormesh(t,f, spec_der)
plt.ylim([0, 10000])
plt.title('Spectral Derivative')

plt.subplot(2,1,2)
plt.pcolormesh(t,f, contour)
plt.title('Contour Plot')

## Librosa Spectral Contrast

In [ ]:
sc_lib = librosa.feature.spectral_contrast(S=stft, sr=fs)
plt.subplots(figsize=(20,10))
plt.pcolormesh(t, np.arange(7), sc_lib)
plt.colorbar()
plt.title('Spectral Contrast')
plt.xlabel('Time (s)')

## Mel Spectrogram

In [ ]:
mel_stft = librosa.feature.melspectrogram(sr=fs, S=stft, n_mels=128, fmax=10000)
mel_f = librosa.mel_frequencies(n_mels=128, fmax=10000)

# Calculate the spectral derivative as the sum of the time derivative and the frequency derivative
dPdt_mel = np.diff(mel_stft, axis=1, append=0)
dPdf_mel = np.diff(mel_stft, axis=0, append=0)
spec_der_mel = dPdt_mel + dPdf_mel

plt.clf()
plt.figure(figsize=(20,10))

plt.subplot(2,2,1)
plt.pcolormesh(t,f, stft)
plt.title('STFT')
plt.colorbar()
plt.ylim([0, 10000])
plt.xlabel('Time (s)')

plt.subplot(2,2,3)
plt.pcolormesh(t, f, spec_der)
plt.title('Spectrogram Derivative')
plt.colorbar()
plt.ylim([0, 10000])
plt.xlabel('Time (s)')
plt.tight_layout()

plt.subplot(2,2,2)
plt.pcolormesh(t,mel_f, mel_stft)
plt.title('Mel Spectrogram')
plt.colorbar()
plt.ylim([0, 10000])
plt.xlabel('Time (s)')

plt.subplot(2,2,4)
plt.pcolormesh(t,mel_f, spec_der_mel)
plt.title('Mel Spectrogram Derivative')
plt.colorbar()
plt.ylim([0, 10000])
plt.xlabel('Time (s)')
plt.tight_layout()